## Preambule

In [541]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import pandas as pd
import json
import plotly

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Parameters

In [542]:
rules = ['PCC', 'ECPC', 'AP']
rulecolors = ['goldenrod', 'tomato', 'forestgreen']
year_of_focus = 2035
TEMP = 1.5

## Paths

In [543]:
path_main = Path("K:/Code/effort-sharing/")
path_data = Path("K:/Data/Data_effortsharing/DataUpdate_ongoing/")
path_figs = path_main / "Figures" / "Paper_FairShares"

## Read data files

In [544]:
xr_dataread = xr.open_dataset(path_data / "startyear_2021/xr_dataread.nc")
xr_traj_yeari = xr.open_dataset(path_data / ("startyear_2021/Aggregated_files/xr_alloc_"+str(year_of_focus)+"_GHG_incl.nc"))
xr_alloc_yeari = xr.open_dataset(path_data / ("startyear_2021/Aggregated_files/xr_alloc_"+str(year_of_focus)+"_GHG_incl.nc"))
xr_total = xr.open_dataset(path_data / "xr_policyscen.nc")

all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)

## Get Cost-optimal output

In [545]:
if TEMP == 1.5:
    Ccat = ['C1']
    figname = "Figure_15C"
    elevscen = "ELV-SSP2-650P-400F"
elif TEMP == 2.0:
    Ccat = ['C3']
    figname = "Figure_2C"
    elevscen = "ELV-SSP2-1150F"

In [546]:
df = pd.read_csv(path_main / "data" / "input" / "plotting_input_data" / "Data_D2.3_vetted_20250519.csv")
df = df[(df.Variable.isin(['Emissions|Kyoto Gases', 'Price|Carbon'])) & (df.Scenario == elevscen)]
df = df.drop(['Scenario', 'Unit'], axis=1)

# Get the regions
regions = []
df['Region'] = df['Region'].replace({'Brazil': 'BRA',
                                     'China': "CHN",
                                     "European Union": "EU",
                                     'India': 'IND',
                                     'Saudi Arabia': 'SAU',
                                     'Japan': 'JPN',
                                     'World': 'WORLD'
                                     })

df = df[df['Region'].isin(['Africa (R10)',
                           'Europe (R10)',
                           'China+ (R10)',
                           'India+ (R10)',
                           'Latin America (R10)',
                           'Middle East (R10)',
                           'North America (R10)',
                           'Other (R10)',
                           'Pacific OECD (R10)',
                           'Reforming Economies (R10)',
                           'Rest of Asia (R10)',

                           'BRA', 'CHN', 'EU', "IND", 'SAU', 'USA', 'JPN',

                           'WORLD'
                           ])]
natives = ['EU', 'USA']

# Set it to xr
dummy = df.melt(id_vars=["Region", "Model", "Variable"], var_name="Time", value_name="Value")
times = np.array(dummy['Time'])
times = np.array(int(i) for i in times)
dummy['Time'] = times
dummy = dummy.set_index(["Region", "Model", "Variable", "Time"])
xr_elev = xr.Dataset.from_dataframe(dummy)
xr_elev = xr_elev.reindex(Time = np.arange(1850, 2101))
xr_elev = xr_elev.interpolate_na(dim="Time", method="linear")

In [547]:
# df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
# df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss'])]
# df_ar6 = df_ar6.reset_index(drop=True)

# df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
# mods = np.array(df_ar6_meta.Model)
# scens = np.array(df_ar6_meta.Scenario)
# modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
# df_ar6_meta['ModelScenario'] = modscens_meta
# df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
# df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(Ccat)]
# df_ar6_meta = df_ar6_meta.reset_index(drop=True)
# ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

# mods = np.array(df_ar6.Model)
# scens = np.array(df_ar6.Scenario)
# modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
# df_ar6['ModelScenario'] = modscens
# df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
# df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

# dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
# dummy['Time'] = np.array(dummy['Time'].astype(int))
# dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
# xr_scen_r = xr.Dataset.from_dataframe(dummy)
# xr_scen_r = xr_scen_r.reindex(Time = np.arange(1850, 2101))
# xr_scen_r = xr_scen_r.interpolate_na(dim="Time", method="linear")

## Compute fractions

World

In [548]:
# df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_Scenarios_Database_World_v1.1.csv")
# df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss', 'Price|Carbon'])]
# df_ar6 = df_ar6.reset_index(drop=True)

# df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
# mods = np.array(df_ar6_meta.Model)
# scens = np.array(df_ar6_meta.Scenario)
# modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
# df_ar6_meta['ModelScenario'] = modscens_meta
# df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
# df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(Ccat)]
# df_ar6_meta = df_ar6_meta.reset_index(drop=True)
# ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

# mods = np.array(df_ar6.Model)
# scens = np.array(df_ar6.Scenario)
# modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
# df_ar6['ModelScenario'] = modscens
# df_ar6['Region'] = 'WORLD'
# df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
# df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

# dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
# dummy['Time'] = np.array(dummy['Time'].astype(int))
# dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
# xr_scen = xr.Dataset.from_dataframe(dummy)
# xr_scen = xr_scen.reindex(Time = np.arange(1850, 2101))
# xr_scen = xr_scen.interpolate_na(dim="Time", method="linear")

Regional

In [549]:
# df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_ISO3_v1.1.csv")
# df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss', 'Price|Carbon'])]
# df_ar6 = df_ar6.reset_index(drop=True)

# df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_ISO3/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
# mods = np.array(df_ar6_meta.Model)
# scens = np.array(df_ar6_meta.Scenario)
# modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
# df_ar6_meta['ModelScenario'] = modscens_meta
# df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
# df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(Ccat)]
# df_ar6_meta = df_ar6_meta.reset_index(drop=True)
# ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

# mods = np.array(df_ar6.Model)
# scens = np.array(df_ar6.Scenario)
# modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
# df_ar6['ModelScenario'] = modscens
# df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
# df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

# dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
# dummy['Time'] = np.array(dummy['Time'].astype(int))
# dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
# xr_scen_r = xr.Dataset.from_dataframe(dummy)
# xr_scen_r = xr_scen_r.reindex(Time = np.arange(1850, 2101))
# xr_scen_r = xr_scen_r.interpolate_na(dim="Time", method="linear")

R-10 regions

In [550]:
# df_ar6 = pd.read_csv("X:/user/dekkerm/Data/IPCC/AR6_R10/AR6_Scenarios_Database_R10_regions_v1.1.csv")
# df_ar6 = df_ar6[df_ar6.Variable.isin(['Emissions|Kyoto Gases', 'Policy Cost|Consumption Loss', 'Price|Carbon'])]
# df_ar6 = df_ar6.reset_index(drop=True)

# df_ar6_meta = pd.read_excel("X:/user/dekkerm/Data/IPCC/AR6_R10/AR6_Scenarios_Database_metadata_indicators_v1.1.xlsx", sheet_name='meta_Ch3vetted_withclimate')
# mods = np.array(df_ar6_meta.Model)
# scens = np.array(df_ar6_meta.Scenario)
# modscens_meta = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
# df_ar6_meta['ModelScenario'] = modscens_meta
# df_ar6_meta = df_ar6_meta[['ModelScenario', 'Category', 'Policy_category']]
# df_ar6_meta = df_ar6_meta[df_ar6_meta.Category.isin(Ccat)]
# df_ar6_meta = df_ar6_meta.reset_index(drop=True)
# ms_meta_refined = np.array(df_ar6_meta.ModelScenario)

# mods = np.array(df_ar6.Model)
# scens = np.array(df_ar6.Scenario)
# modscens = np.array([mods[i]+'|'+scens[i] for i in range(len(scens))])
# df_ar6['ModelScenario'] = modscens
# df_ar6 = df_ar6.drop(['Model', 'Scenario', 'Unit'], axis=1)
# df_ar6 = df_ar6[df_ar6.ModelScenario.isin(np.array(ms_meta_refined))]

# dummy = df_ar6.melt(id_vars=["Variable", "Region", "ModelScenario"], var_name="Time", value_name="Value")
# dummy['Time'] = np.array(dummy['Time'].astype(int))
# dummy = dummy.set_index(["Variable", "Region", "ModelScenario", "Time"])
# xr_scen_r10 = xr.Dataset.from_dataframe(dummy)
# xr_scen_r10 = xr_scen_r10.reindex(Time = np.arange(1850, 2101))
# xr_scen_r10 = xr_scen_r10.interpolate_na(dim="Time", method="linear")

In [551]:
# df_conv = pd.read_excel("X:/user/dekkerm/Data/AR6_regionclasses.xlsx", sheet_name = 'Sheet1')[['ISO', 'region_ar6_10']]
# regs = np.array(df_conv.region_ar6_10)
# regs[regs == 'Africa'] =            'R10AFRICA'
# regs[regs == 'Eastern Asia'] =      'R10CHINA+'
# regs[regs == 'Europe'] =            'R10EUROPE'
# regs[regs == 'Southern Asia'] =     'R10INDIA+'
# regs[regs == 'Latin America and Caribbean'] = 'R10LATIN_AM'
# regs[regs == 'Middle East'] =       'R10MIDDLE_EAST'
# regs[regs == 'North America'] =     'R10NORTH_AM'
# regs[regs == 'Asia-Pacific Developed'] = 'R10PAC_OECD'
# regs[regs == 'Eurasia'] =           'R10REF_ECON'
# regs[regs == 'South-East Asia and developing Pacific'] = 'R10REST_ASIA'
# df_conv['region_ar6_10'] = regs

In [552]:
df_conv = pd.read_excel("X:/user/dekkerm/Data/AR6_regionclasses.xlsx", sheet_name = 'Sheet1')[['ISO', 'region_ar6_10']]
regs = np.array(df_conv.region_ar6_10)
regs[regs == 'Africa'] =            'Africa (R10)'
regs[regs == 'Eastern Asia'] =      'China+ (R10)'
regs[regs == 'Europe'] =            'Europe (R10)'
regs[regs == 'Southern Asia'] =     'India+ (R10)'
regs[regs == 'Latin America and Caribbean'] = 'Latin America (R10)'
regs[regs == 'Middle East'] =       'Middle East (R10)'
regs[regs == 'North America'] =     'North America (R10)'
regs[regs == 'Asia-Pacific Developed'] = 'Pacific OECD (R10)'
regs[regs == 'Eurasia'] =           'Reforming Economies (R10)'
regs[regs == 'South-East Asia and developing Pacific'] = 'Rest of Asia (R10)'
df_conv['region_ar6_10'] = regs

In [553]:
native_regions = natives
xrs = []
for cty in all_countries_iso:
    if cty not in native_regions:
        which_group = np.array(df_conv[df_conv.ISO == cty].region_ar6_10)[0]
        group_members = np.array(df_conv[df_conv.region_ar6_10 == which_group].ISO)
        emis_total = float(xr_dataread.GHG_hist.sel(Region=np.intersect1d(group_members,all_countries_iso), Time=2021).sum(dim='Region'))
        emis_frac = xr_dataread.GHG_hist.sel(Region=cty, Time=2021) / emis_total
        xrry = xr.merge([xr_elev.sel(Region=which_group, Variable=['Emissions|Kyoto Gases'])*emis_frac,
                         #xr_elev.sel(Region=which_group, Variable=['Price|Carbon'])
                         ]).expand_dims({'Region': [cty]})
        xrs.append(xrry)
xr_scen_infilled_r = xr.concat(xrs, dim='Region')

Concatenate

In [554]:
xr_scen_all = xr.merge([xr_elev, xr_scen_infilled_r])

## Data for plots

In [555]:
if TEMP == 2.0:
    set_temp = 2.0
    set_risk = 0.33
elif TEMP == 1.5:
    set_temp = 1.6
    set_risk = 0.5

settings_default = {'Temperature': set_temp,
                   'Risk': set_risk,
                   'NegEmis': 0.5,
                   'NonCO2red': 0.5,
                   'Timing': "Immediate",
                   'Convergence_year': 2050,
                   'Scenario': 'SSP2',
                   'Discount_factor': 0,
                   'Historical_startyear': 1990,}

settings_range = {'Temperature': set_temp,
                   'Risk': set_risk,
                    'NonCO2red':0.5,
                   'Timing': "Immediate",
                   'NegEmis': 0.5,
                   }

In [556]:
ndcs = xr_dataread.GHG_ndc.mean(dim=['Conditionality', 'Ambition'])

In [557]:
xr_policyscen = xr.open_dataset(path_data / "xr_policyscen.nc")
curpols = xr_policyscen.CurPol.sel(Time=year_of_focus).mean(dim='Model')
nzs = xr_policyscen.NetZero.sel(Time=year_of_focus).mean(dim='Model')

In [558]:
path_ctygroups = "X:/user/dekkerm/Data/" + "UNFCCC_Parties_Groups_noeu.xlsx"
df = pd.read_excel(path_ctygroups, sheet_name = "Country groups")
countries_iso = np.array(df["Country ISO Code"])
countries_name = np.array(df["Name"])
countries_iso = np.array(df["Country ISO Code"])
group_eu = countries_iso[np.array(df["EU"]) == 1]

In [559]:
costopts = xr_scen_all.sel(Time=year_of_focus, Variable='Emissions|Kyoto Gases').mean(dim='Model')

## NDC 2035 data

In [560]:
df_ndc_raw = pd.read_excel("X:/user/dekkerm/Data/NDC/NDC tool GHG incl excl LULUCF with Input results.xlsx",
                            sheet_name='results_incl LULUCF', header=5)
regs = df_ndc_raw['country_name']
regs_iso = []
for r in regs:
    wh = np.where(countries_name == r)[0]
    if len(wh) == 0:
        if r == 'United States':
            regs_iso.append('USA')
        elif r == 'EU27':
            regs_iso.append('EU')
        elif r == 'Turkey':
            regs_iso.append('TUR')
        else:
            regs_iso.append(np.nan)
    else:
        regs_iso.append(countries_iso[wh[0]])
regs_iso = np.array(regs_iso)
df_ndc_raw['ISO'] = regs_iso

df_regs = []
df_amb = []
df_con = []
df_emis = []
df_lulucf = []
df_red = []
df_abs = []
df_inv = []
histemis = xr_dataread.GHG_hist.sel(Time=2015)
for r in list(countries_iso) + ['EU']:
    histemis_r = float(histemis.sel(Region=r))
    df_ndc_raw_sub = df_ndc_raw[df_ndc_raw['ISO'] == r]
    if len(df_ndc_raw_sub)>0:
        for cond in ['Unconditional', 'Conditional']:
            for ambition_i, ambition in enumerate(['min', 'max']):
                add = ['Min', 'Max'][ambition_i]
                key = 'NDC_2035_'+cond+'_'+add
                val = float(df_ndc_raw_sub[key])
                red = 1 - val / histemis_r
                abs_jones = histemis_r * (1 - red)
                df_regs.append(r)
                df_amb.append(ambition)
                df_con.append(cond)
                df_emis.append('NDC')
                df_lulucf.append('incl')
                df_red.append(red)
                df_abs.append(abs_jones)
                df_inv.append(val)

dict_ndc = {"Region": df_regs,
            "Ambition": df_amb,
            "Conditionality": df_con,
            "GHG_ndc_red": df_red,
            "GHG_ndc": df_abs,
            "GHG_ndc_inv": df_inv}
df_ndc = pd.DataFrame(dict_ndc)
xr_ndc = xr.Dataset.from_dataframe(df_ndc.set_index(["Region", "Ambition", "Conditionality"]))

C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_7088\1402567247.py:38: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



## Reductions

In [561]:
data_def = xr_traj_yeari.sel(**settings_default)[['PCC', 'AP', 'ECPC']]
data_var = xr_traj_yeari.sel(**settings_range)[['PCC', 'AP', 'ECPC']]
red_mt = data_def
fair_0 = data_var.quantile(0, dim=['Convergence_year', 'Scenario', 'Discount_factor', 'Historical_startyear'])
fair_1 = data_var.quantile(1, dim=['Convergence_year', 'Scenario', 'Discount_factor', 'Historical_startyear'])

co = costopts.Value
co_0 = xr_scen_all.sel(Time=year_of_focus, Variable='Emissions|Kyoto Gases').quantile(0.1, dim='Model')
co_1 = xr_scen_all.sel(Time=year_of_focus, Variable='Emissions|Kyoto Gases').quantile(0.9, dim='Model')

k:\Environments\MyEnv\Lib\site-packages\numpy\lib\nanfunctions.py:1563: RuntimeWarning:

All-NaN slice encountered



In [562]:
costopt_mt = costopts.Value
ndc_est_mt = xr_dataread.GHG_ndc.mean(dim='Ambition').min(dim='Conditionality')
ndc_est_mt2035 = xr_ndc.GHG_ndc.mean(dim='Ambition').min(dim='Conditionality')

emis_2015 = xr_dataread.GHG_hist.sel(Time=2015)
reductions = 1-red_mt/emis_2015
ndc_est = 1-(ndc_est_mt / emis_2015)
ndc_est2035 = 1-(ndc_est_mt2035 / emis_2015)
costopt = 1-costopt_mt/emis_2015

red_0 = 1-fair_0/emis_2015
red_1 = 1-fair_1/emis_2015
costopt_0 = 1-co_0/emis_2015
costopt_1 = 1-co_1/emis_2015

def data(reg): # Reduction targets
    return float(ndc_est.sel(Region=reg)), float(ndc_est2035.sel(Region=reg)), np.array(reductions.sel(Region=reg)[['PCC', 'AP', 'ECPC']].to_array()), float(costopt.sel(Region=reg))
def data_mt(reg): # Absolute emissions levels
    return float(ndc_est_mt.sel(Region=reg)), np.array(red_mt.sel(Region=reg)[['PCC', 'AP', 'ECPC']].to_array()), float(costopt_mt.sel(Region=reg))
def red_minmax(reg, rule): # Reduction targets
    return float(red_0.sel(Region=reg)[rule]), float(red_1.sel(Region=reg)[rule])

## Finance plot

In [563]:
col1 = "#afc50f"#"rgba(255, 65, 54, 1)"
col2 = '#2e5061'#'rgba(61, 153, 112, 1)'
col3 = '#0062a6'
col3a = '#5db2de'
col3b = '#0062a6'
col3c = '#002c4d'

In [564]:
costopt_0.sel(Region='EU')

<xarray.Dataset> Size: 28B
Dimensions:   ()
Coordinates:
    Region    <U2 8B 'EU'
    quantile  float64 8B 0.1
    Time      int32 4B 2015
Data variables:
    Value     float64 8B 0.4666

In [565]:
wd = 0.15

fig = make_subplots(rows=1, cols=1,
                    horizontal_spacing = 0., vertical_spacing=0.02)

fig.update_layout(height=1000, width=1000)
fs=20

# ============ #
# Actual data
# ============ #

# Vertical lines
fig.add_shape(
    col=1, row=1,
    type="line",
    x0=0, x1=0,
    y0=-1e3, y1=1e3,
    line=dict(color="black",  dash='dash',width=3)
)
for x in [-1.25, -1, -0.75, -0.5, -0.25, 0.25, 0.5, 0.75, 1, 1.25]:
    fig.add_shape(
        col=1, row=1,
        type="line",
        x0=x, x1=x,
        y0=-1e3, y1=1e3,
        line=dict(color="silver", width=0.5)
    )
for y in range(11):
    fig.add_shape(
        col=1, row=1,
        type="line",
        x0=-1e3, x1=1e3,
        y0=y-0.45, y1=y-0.45,
        line=dict(color="silver", width=0.5)
    )

# Data
REGS = ['USA', 'EU', 'JPN', 'BRA', 'IDN', 'VNM', 'CHN', 'PAK', 'IND', 'NGA']

ndcs = []
ndcs2 = []
reds = []
for reg_i, reg in enumerate(REGS):
    ndc_estimate, ndc2, reductions_i, costopt_i = data(reg)
    ndcs.append(ndc_estimate)
    ndcs2.append(ndc2)
    reds.append(reductions_i)
    cty = all_regions_names[all_regions_iso==reg][0].replace(' ', '<br>')
    emiscap_2015 = np.round(float(xr_dataread.GHG_hist.sel(Region=reg, Time=2015)/xr_dataread.Population.sel(Region=reg, Time=2015, Scenario='SSP2')),1)
    if reg == 'USA': cty = 'United<br>States'
    fig.add_annotation(x=-0.05, y=reg_i*1.+0.1, align = 'left',
                       text=cty, xref='paper', yref='y1', showarrow=False, font=dict(color='black', size=20))
    fig.add_annotation(x=-0.05, y=reg_i*1.-0.3, align = 'left',
                       text=str(emiscap_2015)+' t CO<sub>2</sub>e/cap', xref='paper', yref='y1', showarrow=False, font=dict(color='black', size=15))

    # WHISKERS
    c0 = -float(costopt_0.sel(Region=reg).Value)
    c1 = -float(costopt_1.sel(Region=reg).Value)
    for c in [c0, c1]:
        if not np.isnan(c):
            fig.add_shape(type="line", col=1, row=1, showlegend=False,
                        line=dict(color=col1, width=2),
                        x0=c, x1=c,
                        y0=-wd/2 + reg_i-wd , y1=wd/2 + reg_i-wd,
            )
    if reg_i ==0: sl = True
    else: sl = False
    if not np.isnan(c0):
        fig.add_shape(type="line", col=1, row=1,
                    line=dict(color=col1, width=4),
                    showlegend=sl,
                    name='Globally cost optimal',
                    x0=c0, x1=c1,
                    y0= reg_i-wd, y1= reg_i-wd,
        )

    # Horizontal lines for each rule
    for i in range(3):
        mn, mx= red_minmax(reg, ['PCC', 'AP', 'ECPC'][i])
        fig.add_trace(go.Scatter(x=[-mn, -mx],
                                y=[reg_i+wd+0.1*i, reg_i+wd+0.1*i],showlegend=False,
                            line=dict(color=[col3a, col3b, col3c][i], width=2),
        ))

    # Horizontal lines between ndcs
    if not np.isnan(ndc2):
        fig.add_trace(go.Scatter(x=[-ndc_estimate, -ndc2],
                                y=[reg_i, reg_i],showlegend=False,
                            line=dict(color=col2, width=2),
        ))


# NDC 2030
fig.add_trace(go.Scatter(
    x=-np.array(ndcs),
    y=np.arange(len(REGS)),
    mode='markers',
    name='NDC 2030',
    showlegend=True,
    marker=dict(
        symbol='square',
        size=12,         # Marker size
        color='silver'     # Marker color
    )
))

# NDC 2035
fig.add_trace(go.Scatter(
    x=-np.array(ndcs2),
    y=np.arange(len(REGS)),
    mode='markers',
    name='NDC 2035',
    showlegend=True,
    marker=dict(
        symbol='square',
        size=12,         # Marker size
        color=col2     # Marker color
    )
))

# rules
for i in range(3):
    fig.add_trace(go.Scatter(
        x=-np.array(reds)[:, i],
        y=np.arange(len(REGS))+wd+0.1*i,
        mode='markers',
        name=['Per capita convergence', 'Ability to pay', 'Equal cumulative per capita'][i],
        showlegend=True,
        marker=dict(
            symbol=['circle', 'diamond', 'star'][i],
            size=12,         # Marker size
            color=[col3a, col3b, col3c][i]     # Marker color
        )
    ))

    # Indicate ends
    for j in range(len(np.array(reds))):
        if -np.array(reds)[j, i] > 1.5:
            val = str(int(100*float(-np.array(reds)[j, i])))
            fig.add_annotation(x=1.5, y=j+wd+0.1*i+0.1, align = 'left',
                            text=val+'% ->', xref='x1', yref='y1', showarrow=False, font=dict(color=[col3a, col3b, col3c][i], size=15))

fig.update_xaxes(row=1, col=1, range=(-1.5, 1.5), tickfont=dict(size=20), side='top',
                    tickvals=[-1.5, -1, -0.5, 0, 0.5,  1, 1.5],
                    ticktext=['150%', '100%', '(reduction)<br>50%',  'GHG emissions<br>incl. land use<br>2015', '(increase)<br>50%', '100%', '150%'])

fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
        legend=dict(
        title='',     # Optional title
        x=0.65, y=0.15,            # Positioning (top-right)
        bgcolor='rgba(255,255,255,0.5)',  # Semi-transparent background
        bordercolor='black',
        borderwidth=0,
        font=dict(size=20)
    ),
)
fig.update_yaxes(row=1, col=1, range=(-0.5, 10
                                      ), showticklabels=False)
fig.write_image("../../../Figures/COMMITTEDpolicybrief/"+figname+".png", scale=5)
fig.write_image("../../../Figures/COMMITTEDpolicybrief/"+figname+".svg", scale=5)
fig.show()